# Load and Geocode Census Rent Data



First load the ACS 2014 1-year median contract rent data, available for download from the Census Bureau's American FactFinder web site. Then parse principle city/state from each metropolitan and micropolitan area name. Then geocode the principle city/state to latitude-longitude. Finally, save to CSV.

In [1]:
import pandas as pd, time, requests, json

In [2]:
# load the ACS rent data and rename the columns to something more intuitive
df = pd.read_csv('data/ACS_14_1YR_B25058_metro_micro/ACS_14_1YR_B25058.csv')
df = df.rename(columns={'GEO.id':'geo_id', 
                        'GEO.id2':'geo_id2', 
                        'GEO.display-label':'msa_name', 
                        'HD01_VD01':'median_contract_rent',
                        'HD02_VD01':'margin_error'})
df.head()

geo_id  geo_id2                   msa_name  median_contract_rent  \
0  310M200US10140    10140    Aberdeen, WA Micro Area                   626   
1  310M200US10180    10180     Abilene, TX Metro Area                   613   
2  310M200US10300    10300      Adrian, MI Micro Area                   556   
3  310M200US10420    10420       Akron, OH Metro Area                   627   
4  310M200US10460    10460  Alamogordo, NM Micro Area                   700   

   margin_error  
0            36  
1            32  
2            23  
3            13  
4           135

In [3]:
# parse principle city/state from MSA name - mostly consistent except for louisville and honolulu
def parse_city_state(msa_name):
    #first get the city name
    city_name = msa_name.split(',')[0]
    city_name = city_name.split('-')[0]
    city_name = city_name.split('/')[0] #Louisville/Jefferson County, KY
    city_name = city_name.replace('Urban ', '') #Urban Honolulu
    
    #now get the state name
    state_name = msa_name.split(',')[1]
    state_name = state_name.strip().split(' ')[0]
    state_name = state_name.split('-')[0]
    
    city_state = '{}, {}'.format(city_name, state_name)
    return city_state

df['city_state'] = df['msa_name'].map(parse_city_state)
df.head()

geo_id  geo_id2                   msa_name  median_contract_rent  \
0  310M200US10140    10140    Aberdeen, WA Micro Area                   626   
1  310M200US10180    10180     Abilene, TX Metro Area                   613   
2  310M200US10300    10300      Adrian, MI Micro Area                   556   
3  310M200US10420    10420       Akron, OH Metro Area                   627   
4  310M200US10460    10460  Alamogordo, NM Micro Area                   700   

   margin_error      city_state  
0            36    Aberdeen, WA  
1            32     Abilene, TX  
2            23      Adrian, MI  
3            13       Akron, OH  
4           135  Alamogordo, NM

In [4]:
# geocode each city/state to lat/long for mapping
def geocode(row):
    pause = 0.5 #pause 0.5 sec between requests
    address = row['city_state']
    print(row.name, end=' ')
    
    time.sleep(pause) #pause for some duration before each request, to not hammer their server
    url = 'http://maps.googleapis.com/maps/api/geocode/json?address={}&sensor=false' #api url with placeholders
    request = url.format(address) #fill in the placeholder with a variable
    response = requests.get(request) #send the request to the server and get the response
    data = json.loads(response.text) #convert the response json string into a dict
    
    if len(data['results']) > 0: #if google was able to geolocate our address, extract lat-long from result
        latitude = data['results'][0]['geometry']['location']['lat']
        longitude = data['results'][0]['geometry']['location']['lng']
        return pd.Series({'latitude':latitude, 'longitude':longitude})
    else:
        print(data) #if we didn't get any results back, what did the response say?

In [5]:
# lat_long comes back as a df with 2 columns, so concatenate it to the original df when done
lat_long = df.apply(geocode, axis=1)
df = pd.concat([df, lat_long], axis=1)
df.head()

0 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 

geo_id  geo_id2                   msa_name  median_contract_rent  \
0  310M200US10140    10140    Aberdeen, WA Micro Area                   626   
1  310M200US10180    10180     Abilene, TX Metro Area                   613   
2  310M200US10300    10300      Adrian, MI Micro Area                   556   
3  310M200US10420    10420       Akron, OH Metro Area                   627   
4  310M200US10460    10460  Alamogordo, NM Micro Area                   700   

   margin_error      city_state   latitude   longitude  
0            36    Aberdeen, WA  46.975371 -123.815722  
1            32     Abilene, TX  32.448736  -99.733144  
2            23      Adrian, MI  41.897547  -84.037166  
3            13       Akron, OH  41.081445  -81.519005  
4           135  Alamogordo, NM  32.899532 -105.960265

In [6]:
# all done - save to csv
df.to_csv('rent_latlong.csv', index=False)